In [5]:
import numpy as np
import pandas as pd
from sklearn.utils import shuffle

# NYC taxi. March, 2016
data = pd.read_csv(filepath_or_buffer='yellow_tripdata_2016-03.csv')
# Initial data dimensions
print(data.shape)
data.head()

(12210952, 19)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,1,2.50,-73.976746,40.765152,1,N,-74.004265,40.746128,1,9.0,0.5,0.5,2.05,0.00,0.3,12.35
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.90,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.00,0.3,15.35
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,2,19.98,-73.782021,40.644810,1,N,-73.974541,40.675770,1,54.5,0.5,0.5,8.00,0.00,0.3,63.80
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,3,10.78,-73.863419,40.769814,1,N,-73.969650,40.757767,1,31.5,0.0,0.5,3.78,5.54,0.3,41.62
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,5,30.43,-73.971741,40.792183,3,N,-74.177170,40.695053,1,98.0,0.0,0.0,0.00,15.50,0.3,113.80


In [6]:
# Choosed features:
data = data[['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
             'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']]
print(data.shape)
data.head()

(12210952, 7)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,1,2016-03-01 00:00:00,2016-03-01 00:07:55,-73.976746,40.765152,-74.004265,40.746128
1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,-73.983482,40.767925,-74.005943,40.733166
2,2,2016-03-01 00:00:00,2016-03-01 00:31:06,-73.782021,40.644810,-73.974541,40.675770
3,2,2016-03-01 00:00:00,2016-03-01 00:00:00,-73.863419,40.769814,-73.969650,40.757767
4,2,2016-03-01 00:00:00,2016-03-01 00:00:00,-73.971741,40.792183,-74.177170,40.695053


In [7]:
# Clear invalid trips, i.e. dropoff earlier than pickup
data = data.loc[data.tpep_pickup_datetime < data.tpep_dropoff_datetime]
data.shape

(12198402, 7)

In [8]:
# Checking data distribution
data.describe()

,VendorID,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
count,1.219840e+07,1.219840e+07,1.219840e+07,1.219840e+07,1.219840e+07
mean,1.531154e+00,-7.287561e+01,4.014585e+01,-7.300812e+01,4.021979e+01
std,4.990285e-01,8.946978e+00,4.928559e+00,8.396655e+00,4.625489e+00
min,1.000000e+00,-1.616987e+02,0.000000e+00,-1.616987e+02,0.000000e+00
25%,1.000000e+00,-7.399174e+01,4.073632e+01,-7.399126e+01,4.073471e+01
50%,2.000000e+00,-7.398161e+01,4.075325e+01,-7.397963e+01,4.075378e+01
75%,2.000000e+00,-7.396678e+01,4.076765e+01,-7.396236e+01,4.076912e+01
max,2.000000e+00,0.000000e+00,6.685682e+01,0.000000e+00,5.079786e+01


In [9]:
# Difference between Min and Max of latitudes and longitudes are abnormally high
# but overall distribution is fine. Need to clean out outliers (Someone's traveling to Africa by taxi xD)
# We can suppose the distribution is normal and keep data in range of [mean-2*std,mean+2*std]

# Calculate means
# Pickup latitude
plat_mean = np.mean(data['pickup_latitude'])
# Pickup longitude
plong_mean = np.mean(data['pickup_longitude'])
# Dropoff latitude
dlat_mean = np.mean(data['dropoff_latitude'])
# Dropoff longitude
dlong_mean = np.mean(data['dropoff_longitude'])
plat_mean, plong_mean, dlat_mean, dlong_mean

(40.145849766934894, -72.87560904146224, 40.21979101957289, -73.00811566130828)

In [10]:
# Calculate stds
# Pickup latitude
plat_std = np.std(data['pickup_latitude'])
# Pickup longitude
plong_std = np.std(data['pickup_longitude'])
# Dropoff latitude
dlat_std = np.std(data['dropoff_latitude'])
# Dropoff longitude
dlong_std = np.std(data['dropoff_longitude'])
plat_std, plong_std, dlat_std, dlong_std

(4.928558813737272, 8.946977428814003, 4.62548910033178, 8.396654436315325)

In [11]:
# Pickup latitude constraints
data = data.loc[(data.pickup_latitude >= (plat_mean-2*plat_std)) &
              (data.pickup_latitude <= (plat_mean+2*plat_std))]
# Pickup longitude constraints
data = data.loc[(data.pickup_longitude >= (plong_mean-2*plong_std)) &
                (data.pickup_longitude <= (plong_mean+2*plong_std))]
# Dropoff latitude constraints
data = data.loc[(data.dropoff_latitude >= (dlat_mean-2*dlat_std)) &
                (data.dropoff_latitude <= (dlat_mean+2*dlat_std))]
# Dropoff longitude constraints
data = data.loc[(data.dropoff_longitude >= (dlong_mean-2*dlong_std)) &
                (data.dropoff_longitude <= (dlong_mean+2*dlong_std))]
data.shape

(12011540, 7)

In [12]:
# Convert string to datetime
data['tpep_pickup_datetime'] = pd.to_datetime(data['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
data['tpep_dropoff_datetime'] = pd.to_datetime(data['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
# Disassemble pickup datetime
data.loc[data.tpep_pickup_datetime.dt.weekday < 5, 'is_weekday'] = 1
data.loc[data.tpep_pickup_datetime.dt.weekday < 5, 'is_weekend'] = 0
data.loc[data.tpep_pickup_datetime.dt.weekday >= 5, 'is_weekday'] = 0
data.loc[data.tpep_pickup_datetime.dt.weekday >= 5, 'is_weekend'] = 1
data['minute_of_the_day'] = data.tpep_pickup_datetime.dt.hour * 60 + data.tpep_pickup_datetime.dt.minute
# Calculate trip time
data['trip_time'] = (data.tpep_dropoff_datetime - data.tpep_pickup_datetime).dt.total_seconds()

# Delete datetime column
data = data.drop('tpep_pickup_datetime', axis=1)
data = data.drop('tpep_dropoff_datetime', axis=1)

data.head()

,VendorID,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,is_weekday,is_weekend,minute_of_the_day,trip_time
0,1,-73.976746,40.765152,-74.004265,40.746128,1.0,0.0,0,475.0
1,1,-73.983482,40.767925,-74.005943,40.733166,1.0,0.0,0,666.0
2,2,-73.782021,40.644810,-73.974541,40.675770,1.0,0.0,0,1866.0
7,1,-73.788773,40.647758,-73.829208,40.712345,1.0,0.0,0,963.0
8,1,-73.958221,40.764641,-73.967896,40.762901,1.0,0.0,0,299.0


In [13]:
# Getting X and y
y = data['trip_time'].values
data = data.drop('trip_time', axis=1)
X = data.loc[:,:].values

# Shuffle and split to training and testing sets
X, y = shuffle(X, y, random_state=19)
offset = int(X.shape[0] * 0.9)
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((10810386, 8), (10810386,), (1201154, 8), (1201154,))

In [16]:
X_train

array([[ 2.00000000e+00, -7.39743500e+01,  4.07836952e+01, ...,
         1.00000000e+00,  0.00000000e+00,  1.38600000e+03],
       [ 1.00000000e+00, -7.39781265e+01,  4.07629471e+01, ...,
         1.00000000e+00,  0.00000000e+00,  8.07000000e+02],
       [ 1.00000000e+00, -7.38730698e+01,  4.07740974e+01, ...,
         1.00000000e+00,  0.00000000e+00,  9.66000000e+02],
       ...,
       [ 2.00000000e+00, -7.39619827e+01,  4.07706757e+01, ...,
         0.00000000e+00,  1.00000000e+00,  6.06000000e+02],
       [ 2.00000000e+00, -7.39696884e+01,  4.07531128e+01, ...,
         1.00000000e+00,  0.00000000e+00,  5.70000000e+02],
       [ 2.00000000e+00, -7.39689484e+01,  4.07583199e+01, ...,
         0.00000000e+00,  1.00000000e+00,  1.50000000e+01]])

In [14]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc

In [15]:
learning_rates = [0.75, 0.9, 0.9, 0.9, 0.9, 0.5, 0.1, 0.5 ]
for learning_rate in learning_rates:
    gb = GradientBoostingClassifier(n_estimators=20, learning_rate = learning_rate, max_features=2, max_depth = 2, random_state = 0)
    gb.fit(X_train, y_train)

ValueError: Unknown label type: 'continuous'